In [1]:
using Serialization, Turing
include("ode_problem.jl");
include("target_probability.jl");
include("bayesian_inference.jl");

[ Info: Precompiling DiffEqBaseTrackerExt [1ba67e42-7aa7-5162-ac9b-c09642cdebbf] (cache misses: wrong dep version loaded (4))
[ Info: Precompiling DistributionsADLazyArraysExt [1640c15e-5bc7-5daf-855c-3cb1bdbdee06] (cache misses: wrong dep version loaded (2))
[ Info: Precompiling SimpleNonlinearSolveTrackerExt [ac004708-0488-5d74-bdd8-3e2743d441a0] (cache misses: wrong dep version loaded (4))
[ Info: Precompiling OptimizationMTKExt [ead85033-3460-5ce4-9d4b-429d76e53be9] (cache misses: wrong dep version loaded (4))
[ Info: Precompiling FastPowerTrackerExt [18c65b70-2546-5492-ac4c-dad4ac5611e8] (cache misses: wrong dep version loaded (4))
[ Info: Precompiling AdvancedHMCOrdinaryDiffEqExt [b79d61f8-5e13-53f1-8c3e-d7d191782ea7] (cache misses: wrong dep version loaded (4))


### Define Target Distribution for Sampling

In [5]:
#first input we need are data points and each point's associated standard deviation
experiment_timecourse = deserialize("outputs/000_processed_active_G_timecourse.dict")
experiment_dose_response = deserialize("outputs/000_processed_active_G_dose_response.dict");
#reshape for input to target probability 
data_points = vcat(experiment_timecourse["response"], experiment_dose_response["response"])
std_dev = vcat(fill(experiment_timecourse["average_error"], length(experiment_timecourse["response"])), 
fill(experiment_dose_response["average_error"], length(experiment_dose_response["response"])))

#next input we need is an ode problem
odesys, u0, tspan, p = return_ode_problem_default_inputs()
odeprob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p);

#also need regularization hyperparameters
#note - must convert from M to molecules to match units
avogadros_constant = 6.022e23 
regularization_hyperparams = Dict("mean" => log10(1.01e-7*avogadros_constant), "std_dev" => 0.8, "lambda" => 1)
regularization_hyperparams_theoretical = Dict("mean" => log10(5.0e-9*avogadros_constant), "std_dev" => 1, "lambda" => 1)

#and ode solver inputs for both timecourse simulation and dose response simulation
solver_inputs_timecourse = return_ode_problem_solver_default_inputs("timecourse")
solver_inputs_dose_response = return_ode_problem_solver_default_inputs("dose_response")

#finally, ligand concentrations for dose respones simulation and normalization
ligand_doses = experiment_dose_response["ligand_stimulation (molecules)"];
normalization_dose = experiment_dose_response["normalize_to_response_at_dose"];
all_ligand_doses = vcat(ligand_doses,normalization_dose);

In [6]:
#target probability definition
target_distribution_regularized = logprob_regularized(data_points, odeprob, std_dev, regularization_hyperparams, solver_inputs_timecourse, 
solver_inputs_dose_response, all_ligand_doses);

target_distribution_regularized_theoretical = logprob_regularized(data_points, odeprob, std_dev, regularization_hyperparams_theoretical, solver_inputs_timecourse, 
solver_inputs_dose_response, all_ligand_doses);

target_distribution_unregularized = logprob_unregularized(data_points, odeprob, std_dev, solver_inputs_timecourse, 
solver_inputs_dose_response, all_ligand_doses);

### Run Affine Invariant Sampler

In [7]:
#affine invariant sampler parameters
n_ensemble = 1
n_walkers = 1000
n_iterations = 1000

#target probability
target_probability = Dict("unregularized"=>target_distribution_unregularized, 
"regularized"=>target_distribution_regularized, "regularized_theoretical"=>target_distribution_regularized_theoretical)

Dict{String, Model{F, argnames, (), (), Targs, Tuple{}, DefaultContext} where {F, argnames, Targs}} with 3 entries:
  "regularized"             => Model{typeof(logprob_regularized), (:data, :odep…
  "regularized_theoretical" => Model{typeof(logprob_regularized), (:data, :odep…
  "unregularized"           => Model{typeof(logprob_unregularized), (:data, :od…

In [16]:
#initial run for both regularized and unregularized 
for i in ["regularized_theoretical", "unregularized", "regularized"]
    affine_invariant_mcmc_firstrun(n_ensemble, n_walkers, n_iterations, target_probability[i], i)
end

┌ Warning: At t=0.31956765289595584, dt was forced below floating point epsilon 5.551115123125783e-17, and step error estimate = 1.1174328294930196. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/hJh6T/src/integrator_interface.jl:623
┌ Warning: At t=0.31956765289595584, dt was forced below floating point epsilon 5.551115123125783e-17, and step error estimate = 1.1174328294930196. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/hJh6T/src/integrator_interface.jl:623
┌ Warning: At t=0.31956765289595584, dt was forced below floating point epsilon 5.551115123125783e-17, and step error estimate = 1.1174328294930196. Aborting. There is either an error in your model sp

In [8]:
start = 31
n_subchains = 40
approach = "regularized"
for i in start:n_subchains
    affine_invariant_mcmc(n_ensemble, n_walkers, n_iterations, target_probability[approach], i, approach)
end

approach = "unregularized"
for i in start:n_subchains
    affine_invariant_mcmc(n_ensemble, n_walkers, n_iterations, target_probability[approach], i, approach)
end

approach = "regularized_theoretical"
for i in start:n_subchains
    affine_invariant_mcmc(n_ensemble, n_walkers, n_iterations, target_probability[approach], i, approach)
end

┌ Warning: At t=242.32597937885015, dt was forced below floating point epsilon 2.842170943040401e-14, and step error estimate = 1.1174328132205966. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/hJh6T/src/integrator_interface.jl:623
┌ Warning: At t=277.64108376658334, dt was forced below floating point epsilon 5.684341886080802e-14, and step error estimate = 1.117433011564668. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/hJh6T/src/integrator_interface.jl:623
┌ Warning: At t=391.6874263898566, dt was forced below floating point epsilon 5.684341886080802e-14, and step error estimate = 1.1174328085906997. Aborting. There is either an error in your model specifi